In [208]:
import os
import dotenv
import requests
import json
from io import BytesIO
import time

from IPython.display import display, Image, HTML
import openai
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

In [18]:
# Access variables
dotenv.load_dotenv('.env')

stable_diffusion_api_key = os.getenv('STABLE_DIFFUSION_API_KEY')
openai.api_key = os.getenv("OPENAI_API_KEY")

## Table of Contents (outdated):

IMAGE
* [Fine-tuning model](#Fine-tuning-model)
    * [Check training status](#Check-training-status)
* [Generate new image](#Generate-new-image)
* [Images for prompts](#Images-for-prompts)
    * [Prompt 3](#Prompt-3)
 

### Prompt tuning resources:
- [https://stable-diffusion-art.com/prompt-guide/](https://stable-diffusion-art.com/prompt-guide/)
- [https://midlibrary.io/styles/metal-gear-solid](https://midlibrary.io/styles/metal-gear-solid)


### Keyword categories:
1. **Subject**: The subject is what you want to see in the image. A common mistake is not writing enough about the subjects.
2. **Medium**: Medium is the material used to make artwork. Some examples are illustration, oil painting, 3D rendering, and photography. Medium has a strong effect because one keyword alone can dramatically change the style.
3. **Style**: The style refers to the artistic style of the image. Examples include impressionist, surrealist, pop art, etc.
4. **Artist**: Artist names are strong modifiers. They allow you to dial in the exact style using a particular artist as a reference. It is also common to use multiple artist names to blend their styles. Now let’s add Stanley Artgerm Lau, a superhero comic artist, and Alphonse Mucha, a portrait painter in the 19th century.
5. **Website**: Niche graphic websites such as Artstation and Deviant Art aggregate many images of distinct genres. Using them in a prompt is a sure way to steer the image toward these styles.
6. **Resolution**: Resolution represents how sharp and detailed the image is. Let’s add keywords highly detailed and sharp focus.
7. **Additional details**: Additional details are sweeteners added to modify an image. We will add sci-fi, stunningly beautiful and dystopian to add some vibe to the image.
8. **Color**: You can control the overall color of the image by adding color keywords. The colors you specified may appear as a tone or in objects.
9. **Lighting**: Any photographer would tell you lighting is a key factor in creating successful images. Lighting keywords can have a huge effect on how the image looks. Let’s add cinematic lighting and dark to the prompt.

In [19]:
message_prompt = """Give me a birthday card message with an astrology prediction 
for the upcoming year for a person born on 3/24. Their zodiac sign is Aries. 
Make it personal and keep it around one paragraph in length. Do not mention their 
birthday in the response. Focus on the astrology prediction for the upcoming year. 
Do not include salutations or complimentary closings."""

In [26]:
message_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a birthday card writer and astrology predictor."},
        {"role": "user", "content": message_prompt},
    ]
)

In [27]:
message_response.choices[0].message.content

'Happy birthday to an adventurous and passionate Aries! As the stars align for you this year, get ready for an exciting journey filled with new beginnings, personal growth, and abundant opportunities. Your courageous nature will be your greatest strength, pushing you to step out of your comfort zone and embrace bold challenges. The universe will fuel your ambition, allowing you to manifest your dreams and achieve remarkable success. However, remember to balance your fiery energy with patience and diplomacy, as it will be key to maintaining harmonious relationships. Trust your instincts, trust the universe, and prepare for a year of extraordinary possibilities.'

In [20]:
df = pd.DataFrame()

# Fine-tuning model
Note: images must be cropped to 512 x 512 pixels

Model Output:
```
{
    "status":"success",
    "messege":"deploying_gpu",
    "data":"it will take upto 45 minutes.",
    "training_id":"2utUQxLpZbNoAJh05TWMvcHDq",
    "model_id":"2utUQxLpZbNoAJh05TWMvcHDq"
}
```

In [13]:
url = "https://stablediffusionapi.com/api/v3/fine_tune"

payload = json.dumps({
  "key": stable_diffusion_api_key,
  "instance_prompt": "photo of adhik person",
  "class_prompt": "photo of person",
  "base_model_id": "portraitplus-diffusion",
  "images": [
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_1.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_2.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_3.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_4.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_5.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_6.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_7.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_8.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_9.png",
    "https://birthdate-public.s3.amazonaws.com/user-uploads/train_10.png"
  ],
  "seed": "0",
  "training_type": "men",
  "max_train_steps": "2000",
  "webhook": ""
})

headers = {
  'Content-Type': 'application/json'
}

training_response = requests.request("POST", url, headers=headers, data=payload)
training_response_json = json.loads(response.text)
print(training_response_json)

{"status":"success","messege":"deploying_gpu","data":"it will take upto 45 minutes.","training_id":"2utUQxLpZbNoAJh05TWMvcHDq","model_id":"2utUQxLpZbNoAJh05TWMvcHDq"}


In [43]:
# model_id = response_json["model_id"]
model_id = "2utUQxLpZbNoAJh05TWMvcHDq"
# training_id = response_json["training_id"]
training_id = "2utUQxLpZbNoAJh05TWMvcHDq"

# Check training status

In [9]:
url = f"https://stablediffusionapi.com/api/v3/fine_tune_status/{model_id}"

payload = json.dumps({
 "key": stable_diffusion_api_key
})

headers = {
  'Content-Type': 'application/json'
}

training_status_response = requests.request("POST", url, headers=headers, data=payload)

In [10]:
training_status_response_json = json.loads(training_status_response.text)

In [11]:
print(training_status_response_json["message"])
print(training_status_response_json["eta"])
print(training_status_response_json["model_download_link"])
print(training_status_response_json["test_image"])

Model is ready to use
0 minutes
https://pub-3626123a908346a7a8be8d9295f44e26.r2.dev/trained-models/2utUQxLpZbNoAJh05TWMvcHDq.tar.gz
https://pub-3626123a908346a7a8be8d9295f44e26.r2.dev/generations/2utUQxLpZbNoAJh05TWMvcHDq.png


# Generate new image

In [149]:
model_config = {
  "key": stable_diffusion_api_key,
  "model_id": model_id,
  "prompt": prompt_3,
  "negative_prompt": negative_prompt,
  "width": "1024",
  "height": "728",
  "samples": "1",
  "num_inference_steps": "30",
  "safety_checker": "no",
  "enhance_prompt": "yes",
  "seed": None,
  "guidance_scale": 7.5,
  "multi_lingual": "no",
  "panorama": "no",
  "self_attention": "yes",
  "upscale": "yes",
  "embeddings_model": None,
  "lora_model": None,
  "tomesd": "yes",
  "use_karras_sigmas": "yes",
  "vae": None,
  "lora_strength": None,
  "scheduler": "UniPCMultistepScheduler",
  "webhook": None,
  "track_id": None
}

payload = json.dumps(model_config)

In [230]:
headers = {
  'Content-Type': 'application/json'
}

def generate_img(prompt, config_override):
    options = {"prompt": prompt}
    config = json.dumps({**config_override, **options})  # Merge dictionaries using dictionary unpacking
    response = requests.request("POST", "https://stablediffusionapi.com/api/v4/dreambooth", headers=headers, data=config)
    response_json = json.loads(response.text)
    return response_json



def compare(*prompts, options={}):
    combined_config = {**model_config, **options}
    image_urls = []  # List to hold the URLs of generated images

    # Ensure there is at least one prompt
    if not prompts:
        raise ValueError("At least one prompt is required")

    # Base prompt is the first prompt
    base_prompt = prompts[0]
    print(f"Base prompt: {base_prompt}")

    first_image = generate_img(base_prompt, combined_config)["future_links"][0]

    for i, additional_prompt in enumerate(prompts[1:], start=1):
        # Build up the prompt by combining base_prompt with all preceding prompts
        combined_prompt = f"{base_prompt}, {', '.join(prompts[1:i+1])}"
        print(f"Prompt #{i+1}: {combined_prompt}")
        response_json = generate_img(combined_prompt, combined_config)
        image_url = response_json["future_links"][0]
        image_urls.append(image_url)

    return [first_image] + image_urls  # Return the base image URL and all other image URLs


# def display_images(image_urls):
#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
#     }

#     def fetch_image(url):
#         response = requests.get(url, headers=headers)
#         return Image(data=BytesIO(response.content).getvalue(), width=300)
    
#     # If image_urls is a single string, make it a list
#     if isinstance(image_urls, str):
#         image_urls = [image_urls]
    
#     images = [fetch_image(url) for url in image_urls]  # Fetch all images
#     display(*images)  # Display all images

def display_images(image_urls):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    def fetch_image(url):
        response = requests.get(url, headers=headers)
        # Return an HTML img element
        return f'<img src="{url}" style="display:inline-block; margin:1px; width:300px;" />'

    # If image_urls is a single string, make it a list
    if isinstance(image_urls, str):
        image_urls = [image_urls]

    images_html = [fetch_image(url) for url in image_urls]  # Fetch all images
    # Join all img elements into a single string of HTML and display it
    display(HTML(''.join(images_html)))


In [191]:
urls = compare("adhik", "digital art", "motorcycle")

Image #1: adhik digital art
Image #2: adhik digital art motorcycle


In [194]:
display_images(urls_1)

In [210]:
imgs_2 = compare("adhik", "digital art", "motorcycle")

In [211]:
imgs_2

['https://cdn2.stablediffusionapi.com/generations/0-8896e79a-6587-4c2b-808f-2de8ab9a845f.png',
 'https://cdn2.stablediffusionapi.com/generations/0-2c97647e-0bb4-4ef1-acaf-fedf06b508bc.png',
 'https://cdn2.stablediffusionapi.com/generations/0-f2cfb4c1-ecb4-4470-b487-0c67d25b81d5.png']

In [213]:
display_images(imgs_2)

In [231]:
imgs_3 = compare("adhik", "sun", "nature", "trees")
imgs_3

Base prompt: adhik
Prompt #2: adhik, sun
Prompt #3: adhik, sun, nature
Prompt #4: adhik, sun, nature, trees


['https://cdn2.stablediffusionapi.com/generations/0-94c8ec07-4263-4ec1-86af-9c01f98b35d5.png',
 'https://cdn2.stablediffusionapi.com/generations/0-2ffedd53-792a-4edd-bbba-369f3ebc907b.png',
 'https://cdn2.stablediffusionapi.com/generations/0-902e7310-d7ff-4a01-90d6-f32c32fc07af.png',
 'https://cdn2.stablediffusionapi.com/generations/0-a46092e1-94dc-4e6f-ac93-1dc4c2fd1c8f.png']

In [236]:
display_images(imgs_3)

In [13]:
# PROMPTS

original_prompt = """actual 8K portrait photo of adhik person, \
portrait, happy colors, bright eyes, clear eyes, warm smile, \
smooth soft skin, big dreamy eyes, beautiful intricate colored \
hair, symmetrical, anime wide eyes, soft lighting, detailed face, \
by makoto shinkai, stanley artgerm lau, wlop, rossdraws, concept art, \
digital painting, looking into camera"""

prompt_1 = """A whimsical illustration of Adhik joyfully experiencing \
his zodiac prediction for the year: [insert zodiac prediction here]. \
Captured in a serene setting with soft lighting accentuating his warm smile \
and bright, dreamy eyes. The portrait, reminiscent of a blend between Makoto \
Shinkai and Stanley Artgerm Lau's art styles, showcases a hopeful Adhik, with \
intricate colored hair, gazing optimistically towards the sky, embodying the \
essence of his astrological forecast. The detailed digital painting vividly \
portrays the harmonious colors and emotions of the moment, as Adhik stands amidst \
a gently blooming garden, symbolizing the promising year ahead."""

prompt_2 = """An enchanting depiction of Adhik embarking on a daring adventure \
symbolizing his Aries spirit. The image captures a courageous Adhik stepping out \ 
from the familiar into a realm filled with new beginnings and endless opportunities, \ 
embodying the essence of personal growth. With a determined gaze and a heart full \
of ambition, he stands at the threshold of a vibrant new world that beckons. The soft \
lighting illuminates his path, highlighting the excitement in his big, dreamy eyes \
and the warm, reassuring smile on his face. His intricate colored hair flows gracefully \
in the wind, adding a whimsical touch to this moment of remarkable potential. The art \
style, a blend of Makoto Shinkai’s ethereal scenery and Stanley Artgerm Lau's evocative \
character design, portrays a blend of courage, anticipation, and the promise of a fulfilling \
year ahead. Every element of this digital painting, from Adhik’s expressive stance to the \
expansive horizon that lays ahead, resonates with the adventurous and passionate energy of \
Aries, promising a year of extraordinary possibilities."""

prompt_3 = """actual 8K portrait photo of adhik person, \
action photo, bright eyes, clear eyes, warm smile, determined eyes, \
beautiful intricate colored hair, symmetrical, anime wide eyes, soft lighting, \
detailed face, by Paul Laffoley, Bess Hamiti, Matt Molloy, Amy Giacomelli, \
concept art, digital painting, looking into camera, birthday card"""

negative_prompt = """painting, extra fingers, mutated hands, poorly drawn hands, \
poorly drawn face, deformed, ugly, blurry, bad anatomy, bad proportions, extra limbs, \
cloned face, skinny, glitchy, double torso, extra arms, extra hands, mangled fingers, \
missing lips, ugly face, distorted face, extra legs, anime, naked, no shirt"""


In [45]:
url = "https://stablediffusionapi.com/api/v4/dreambooth"

headers = {
  'Content-Type': 'application/json'
}

gen_response = requests.request("POST", url, headers=headers, data=payload)
gen_response_json = json.loads(gen_response.text)
print(gen_response_json)

{'status': 'processing', 'tip': 'Get 20x faster image generation using enterprise plan. Click here : https://stablediffusionapi.com/enterprise', 'tip_1': 'Your image is processing in background, you can get this image using fetch API: here is link for fetch api : https://stablediffusionapi.com/docs/community-models-api-v4/dreamboothfetchqueimg', 'eta': 8.238667458754739, 'messege': 'Try to fetch request after seconds estimated', 'webhook_status': '', 'fetch_result': 'https://stablediffusionapi.com/api/v3/dreambooth/fetch/51707832', 'id': 51707832, 'output': [], 'meta': {'prompt': 'portrait+ style actual 8K portrait photo of adhik person, action photo, bright eyes, clear eyes, warm smile, determined eyes, beautiful intricate colored hair, symmetrical, anime wide eyes, soft lighting, detailed face, by Paul Laffoley, Bess Hamiti, Matt Molloy, Amy Giacomelli, concept art, digital painting, looking into camera, birthday card hyperrealistic, full body, detailed clothing, highly detailed, cin

In [46]:
fetch_url = gen_response_json["fetch_result"]

payload = json.dumps({
  "key": stable_diffusion_api_key,
})

headers = {
  'Content-Type': 'application/json'
}

fetch_response = requests.request("POST", fetch_url, headers=headers, data=payload)
fetch_response_json = json.loads(gen_response.text)
print(fetch_response_json)

{'status': 'processing', 'tip': 'Get 20x faster image generation using enterprise plan. Click here : https://stablediffusionapi.com/enterprise', 'tip_1': 'Your image is processing in background, you can get this image using fetch API: here is link for fetch api : https://stablediffusionapi.com/docs/community-models-api-v4/dreamboothfetchqueimg', 'eta': 8.238667458754739, 'messege': 'Try to fetch request after seconds estimated', 'webhook_status': '', 'fetch_result': 'https://stablediffusionapi.com/api/v3/dreambooth/fetch/51707832', 'id': 51707832, 'output': [], 'meta': {'prompt': 'portrait+ style actual 8K portrait photo of adhik person, action photo, bright eyes, clear eyes, warm smile, determined eyes, beautiful intricate colored hair, symmetrical, anime wide eyes, soft lighting, detailed face, by Paul Laffoley, Bess Hamiti, Matt Molloy, Amy Giacomelli, concept art, digital painting, looking into camera, birthday card hyperrealistic, full body, detailed clothing, highly detailed, cin

{'status': 'processing', 'tip': 'Get 20x faster image generation using enterprise plan. Click here : https://stablediffusionapi.com/enterprise', 'tip_1': 'Your image is processing in background, you can get this image using fetch API: here is link for fetch api : https://stablediffusionapi.com/docs/community-models-api-v4/dreamboothfetchqueimg', 'eta': 21.20322994262903, 'messege': 'Try to fetch request after seconds estimated', 'webhook_status': '', 'fetch_result': 'https://stablediffusionapi.com/api/v3/dreambooth/fetch/51661231', 'id': 51661231, 'output': [], 'meta': {'prompt': "portrait+ style A whimsical illustration of Adhik joyfully experiencing his zodiac prediction for the year: [insert zodiac prediction here]. Captured in a serene setting with soft lighting accentuating his warm smile and bright, dreamy eyes. The portrait, reminiscent of a blend between Makoto Shinkai and Stanley Artgerm Lau's art styles, showcases a hopeful Adhik, with intricate colored hair, gazing optimistically towards the sky, embodying the essence of his astrological forecast. The detailed digital painting vividly portrays the harmonious colors and emotions of the moment, as Adhik stands amidst a gently blooming garden, symbolizing the promising year ahead. hyperrealistic, full body, detailed clothing, highly detailed, cinematic lighting, stunningly beautiful, intricate, sharp focus, f/1. 8, 85mm, (centered image composition), (professionally color graded), ((bright soft diffused light)), volumetric fog, trending on instagram, trending on tumblr, HDR 4K, 8K", 'model_id': '2utUQxLpZbNoAJh05TWMvcHDq', 'negative_prompt': 'painting, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, deformed, ugly, blurry, bad anatomy, bad proportions, extra limbs, cloned face, skinny, glitchy, double torso, extra arms, extra hands, mangled fingers, missing lips, ugly face, distorted face, extra legs, anime(child:1.5), ((((underage)))), ((((child)))), (((kid))), (((preteen))), (teen:1.5) ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, disfigured, deformed, body out of frame, bad anatomy, watermark, signature, cut off, low contrast, underexposed, overexposed, bad art, beginner, amateur, distorted face, blurry, draft, grainy', 'scheduler': 'UniPCMultistepScheduler', 'safety_checker': 'no', 'W': 1400, 'H': 1000, 'guidance_scale': 7.5, 'seed': 4078858537, 'steps': 20, 'n_samples': 1, 'full_url': 'no', 'tomesd': 'yes', 'upscale': 'no', 'multi_lingual': 'no', 'panorama': 'no', 'self_attention': 'no', 'use_karras_sigmas': 'yes', 'embeddings': None, 'vae': None, 'lora': None, 'lora_strength': 1, 'clip_skip': 1, 'temp': 'no', 'base64': 'no', 'file_prefix': 'e651eaaf-1c19-46a6-aa84-60a885688278.png'}, 'future_links': ['https://cdn2.stablediffusionapi.com/generations/0-e651eaaf-1c19-46a6-aa84-60a885688278.png']}

# Images for prompts

### Prompt 3

In [109]:
print(prompt_3)

actual 8K portrait photo of adhik person, action photo, bright eyes, clear eyes, warm smile, big dreamy eyes, beautiful intricate colored hair, symmetrical, anime wide eyes, soft lighting, detailed face, by Paul Laffoley, Bess Hamiti, Matt Molloy, Amy Giacomelli, Metal Gear Solid, concept art, digital painting, looking into camera


In [123]:
#Image(url='https://cdn2.stablediffusionapi.com/generations/0-6412182c-aa83-4df6-82ad-36917ae1fba6.png') 

In [21]:
#Image(url='https://cdn2.stablediffusionapi.com/generations/0-66a336d3-98f2-480e-98ea-734a911c86df.png') 

In [22]:
#Image(url='https://cdn2.stablediffusionapi.com/generations/0-091e0403-b123-47a7-87bc-40c4af1faafd.png') 

In [ ]:
prompt = "adhik"
